這份 Notebook 示範 Google Gemini API 的呼叫方式


## 設定 Gemini API Key 變數

請點開左側欄的Key符號，就可以設定 Secret。請設定 gemini_api_key

In [1]:
from google.colab import userdata
gemini_api_key = userdata.get('gemini_api_key')

🔥🔥🔥 我們主要使用 requests 和 json library

google 有出 python 套件 https://pypi.org/project/google-generativeai/

但是這裡我的目標聽眾包括非 Python 的工程師，因此我只用最最基本的 HTTP library，相信你很容易就可以替換成不同程式語言的 HTTP library。

In [2]:
import requests
import json # 這有兩個方法 dumps (物件轉字串) 跟 loads (字串轉物件)
from pprint import pp # 為了印出來漂亮

API 文件: https://ai.google.dev/tutorials/rest_quickstart

In [5]:
def get_completion(messages, model="gemini-1.0-pro", temperature=0, max_tokens=1000):
  payload = { "contents": messages,
              "generationConfig": { "temperature": temperature, "maxOutputTokens": max_tokens },
              "safetySettings": [
                {
                  "category": "HARM_CATEGORY_HARASSMENT",
                  "threshold": "BLOCK_NONE"
                },
                {
                  "category": "HARM_CATEGORY_HATE_SPEECH",
                  "threshold": "BLOCK_NONE"
                },
                {
                  "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                  "threshold": "BLOCK_NONE"
                },
                {
                  "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                  "threshold": "BLOCK_NONE"
                }]
             }
  response = requests.post(f'https://generativelanguage.googleapis.com/v1beta/models/{model}:generateContent?key={gemini_api_key}', data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["candidates"][0]["content"]["parts"][0]['text']
  else :
    return obj["error"]

In [ ]:
user_message = "什麼是 Python 語言? 請用台灣繁體中文" # 可以改任意問題

messages = [
  { "role": "user", "parts": [{ "text": user_message }] }
]

response = get_completion(messages, model="gemini-1.0-pro", temperature=0.2)
pp(response)

('Python 是一種廣泛使用的程式語言，以其易於學習和使用而聞名。它是一種直譯式語言，這表示它在執行前不需要編譯成機器碼。\n'
 '\n'
 'Python 具有以下特點：\n'
 '\n'
 '* **易於學習：**Python 的語法簡單明瞭，即使是初學者也能快速上手。\n'
 '* **多功能：**Python 可用於各種應用程式，包括網頁開發、資料科學、機器學習和自動化。\n'
 '* **開放原始碼：**Python 是免費且開放原始碼的，這表示任何人都可以使用、修改和分發它。\n'
 '* **大型社群：**Python 擁有龐大的社群，提供豐富的資源、文件和支援。\n'
 '* **廣泛的套件庫：**Python 有大量的套件庫可用，可擴充其功能並簡化開發。\n'
 '\n'
 'Python 適合各種開發人員，包括初學者、經驗豐富的程式設計師和資料科學家。它廣泛用於以下領域：\n'
 '\n'
 '* 網頁開發\n'
 '* 資料分析和視覺化\n'
 '* 機器學習和人工智慧\n'
 '* 自動化和腳本編寫\n'
 '* 科學計算和數值模擬')


## Temperature 溫度

https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini

* 範圍是: 0.0 - 1.0
* Default for gemini-pro: 0.9
* Default for gemini-pro-vision: 0.4
* OpenAI 是 0.0 - 2.0

In [ ]:
user_message = "小明愛吃什麼? "

messages = [
  { "role": "user", "parts": [{ "text": user_message }] }
]

response = get_completion(messages, temperature=1)  # 可以改看看溫度
print(response)

此處並無提供有關小明喜愛食物的資訊。


## 連續對話多 messages 的場景

In [ ]:
messages=[
     { "role": "user", "parts": [{ "text": "誰贏得2013年的世界棒球經典賽冠軍?" }] }
  ]
response = get_completion(messages, temperature=0.3)
print(response)

多明尼加共和國


接著要繼續對話時，你得把 AI 回覆的訊息，放在 role: model 裡面一起傳給 OpenAI。因為 API 是無狀態的(Stateless)

In [ ]:
messages=[
      { "role": "user", "parts": [{ "text": "誰贏得2013年的世界棒球經典賽冠軍?" }] },
      { "role": "model", "parts": [{ "text": "多明尼加共和國" }] },
      { "role": "user", "parts": [{ "text": "那2017年呢?" }] }
]
response = get_completion(messages, temperature=0.3)
print(response)

美國


### 模型的幻覺現象 Hallucination

In [ ]:
messages=[
      { "role": "user", "parts": [{ "text": "誰贏得2013年的世界棒球經典賽冠軍?" }] },
      { "role": "model", "parts": [{ "text": "多明尼加共和國" }] },
      { "role": "user", "parts": [{ "text": "那2017年呢?" }] },
      { "role": "model", "parts": [{ "text": "美國" }] },
      { "role": "user", "parts": [{ "text": "那2018年呢?" }] }
]
response = get_completion(messages, temperature=0.3)
print(response)

美國


In [ ]:
messages=[
      { "role": "user", "parts": [{ "text": "誰贏得2013年的世界棒球經典賽冠軍?" }] },
      { "role": "model", "parts": [{ "text": "多明尼加共和國" }] },
      { "role": "user", "parts": [{ "text": "那2017年呢?" }] },
      { "role": "model", "parts": [{ "text": "美國" }] },
      { "role": "user", "parts": [{ "text": "那2018年呢? 如果沒舉辦，請回答沒舉辦" }] }
]

response = get_completion(messages, temperature=0.3)
print(response)

沒舉辦


## 多模態 image 影像解讀

圖片用 base64 編碼後上傳

In [ ]:
# 範例圖片
!wget https://ihower.tw/data/images/korea.jpg

--2024-03-04 12:35:30--  https://ihower.tw/data/images/korea.jpg
Resolving ihower.tw (ihower.tw)... 172.104.102.228
Connecting to ihower.tw (ihower.tw)|172.104.102.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 492922 (481K) [image/jpeg]
Saving to: ‘korea.jpg’

korea.jpg           100%[===================>] 481.37K  1.11MB/s    in 0.4s    

2024-03-04 12:35:31 (1.11 MB/s) - ‘korea.jpg’ saved [492922/492922]



In [ ]:
import base64

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
messages = [
      { "role": "user", "parts": [
          { "text": "描述這張照片" },
          { "inlineData": {
                    "mimeType": "image/jpeg",
                    "data": encode_image("korea.jpg")
                  }
                }
          ] },

]

get_completion(messages, model="gemini-pro-vision")

' 這張照片是在韓國的一家咖啡店外拍的。照片中有一塊立牌，上面寫著「팥빙수 100% 국내산」幾個字。這家咖啡店似乎在宣傳他們使用100%韓國國產的食材來製作刨冰。'

In [ ]:
!wget https://ihower.tw/data/images/poster.jpg

--2024-03-04 12:35:55--  https://ihower.tw/data/images/poster.jpg
Resolving ihower.tw (ihower.tw)... 172.104.102.228
Connecting to ihower.tw (ihower.tw)|172.104.102.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 324916 (317K) [image/jpeg]
Saving to: ‘poster.jpg’

poster.jpg          100%[===================>] 317.30K   859KB/s    in 0.4s    

2024-03-04 12:35:55 (859 KB/s) - ‘poster.jpg’ saved [324916/324916]



In [ ]:
messages = [
      { "role": "user", "parts": [
          { "text": "請擷取這張海報，擷取出 1.標題 2. 活動時間。only response json format:" },
          { "inlineData": {
                    "mimeType": "image/jpeg",
                    "data": encode_image("poster.jpg")
                  }
                }
          ] },

]

result = get_completion(messages, model="gemini-pro-vision")
print(result)

 ```json
{
  "title": "大語言模型 LLM 應用 開發工作坊",
  "time": "112年10月24日 09:30~17:30"
}
```


In [ ]:
!wget https://ihower.tw/data/images/card.jpg

--2024-03-04 12:36:00--  https://ihower.tw/data/images/card.jpg
Resolving ihower.tw (ihower.tw)... 172.104.102.228
Connecting to ihower.tw (ihower.tw)|172.104.102.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 270858 (265K) [image/jpeg]
Saving to: ‘card.jpg’

card.jpg            100%[===================>] 264.51K   826KB/s    in 0.3s    

2024-03-04 12:36:01 (826 KB/s) - ‘card.jpg’ saved [270858/270858]



In [ ]:
messages = [
      { "role": "user", "parts": [
          { "text": "請從擷取這張名片。only response json format:" },
          { "inlineData": {
                    "mimeType": "image/jpeg",
                    "data": encode_image("card.jpg")
                  }
                }
          ] },

]

result = get_completion(messages, model="gemini-pro-vision")
print(result)

 ```json
{
  "name": "張文錦",
  "title": null,
  "company": "愛好資訊科技有限公司",
  "phone": "0921056780",
  "email": "ihower@aihao.tw",
  "website": "https://aihao.tw",
  "address": "新竹市高翠路202號13號1F"
}
```


## 中文 OCR 有比 OpenAI 好一點，但還是不夠好

In [ ]:
!wget https://ihower.tw/data/images/number.jpg

--2024-03-04 12:36:06--  https://ihower.tw/data/images/number.jpg
Resolving ihower.tw (ihower.tw)... 172.104.102.228
Connecting to ihower.tw (ihower.tw)|172.104.102.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102598 (100K) [image/jpeg]
Saving to: ‘number.jpg’

number.jpg          100%[===================>] 100.19K   450KB/s    in 0.2s    

2024-03-04 12:36:07 (450 KB/s) - ‘number.jpg’ saved [102598/102598]



In [ ]:
messages = [
      { "role": "user", "parts": [
          { "text": "請告訴畫面中的驗證碼數字?" },
          { "inlineData": {
                    "mimeType": "image/jpeg",
                    "data": encode_image("number.jpg")
                  }
                }
          ] },

]

result = get_completion(messages, model="gemini-pro-vision")
print(result)

 6450


## 列出可用的模型

In [6]:
response = requests.get(f'https://generativelanguage.googleapis.com/v1beta/models?key={gemini_api_key}' )
result = json.loads(response.text)
result["models"]

[{'name': 'models/chat-bison-001',
  'version': '001',
  'displayName': 'PaLM 2 Chat (Legacy)',
  'description': 'A legacy text-only model optimized for chat conversations',
  'inputTokenLimit': 4096,
  'outputTokenLimit': 1024,
  'supportedGenerationMethods': ['generateMessage', 'countMessageTokens'],
  'temperature': 0.25,
  'topP': 0.95,
  'topK': 40},
 {'name': 'models/text-bison-001',
  'version': '001',
  'displayName': 'PaLM 2 (Legacy)',
  'description': 'A legacy model that understands text and generates text as an output',
  'inputTokenLimit': 8196,
  'outputTokenLimit': 1024,
  'supportedGenerationMethods': ['generateText',
   'countTextTokens',
   'createTunedTextModel'],
  'temperature': 0.7,
  'topP': 0.95,
  'topK': 40},
 {'name': 'models/embedding-gecko-001',
  'version': '001',
  'displayName': 'Embedding Gecko',
  'description': 'Obtain a distributed representation of a text.',
  'inputTokenLimit': 1024,
  'outputTokenLimit': 1,
  'supportedGenerationMethods': ['embedT

In [11]:
user_message = "什麼是 Gemini ?"

messages = [
  { "role": "user", "parts": [{ "text": user_message }] }
]

response = get_completion(messages, model="gemini-1.5-pro-latest", temperature=0.2)
pp(response)

('我是 Gemini，一個由 Google '
 '訓練的大型語言模型。我可以理解並生成人類語言，協助您完成各種任務，例如寫作、翻譯和回答您的問題。請問您想讓我做些什麼呢？ \n')
